# Laboratorio 2
## Universidad del Valle de Guatemala <br> Facultas de Ingeniería
#### Departamento de Ciencias de la Computación <br> Deep Learning y Sistemas Inteligentes - Sección 20
#### Grupo 12  
Cristian Laynez, Jeyner Arango

### Objetivo de la Red

### Implementación de Redes

In [96]:
# Paquetes a utilizar
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, regularizers

# Cargar la base de datos
dataset = pd.read_csv('movie_statistic_dataset.csv')

# eliminar las filas con valores faltantes en la columna de destino
dataset = dataset.dropna(subset=['Worldwide gross $'])

# Indexar 'movie_title'
dataset.set_index('movie_title', inplace=True)

#runtime_minutes
dataset['runtime_minutes'] = dataset['runtime_minutes'].astype(int)

#director_name
#Es mejor convertir los directores faltantes a NaN (No es un número)
dataset['director_name'].replace('-', np.nan, inplace=True)

#production_date: AAAA-MM-DD
#Como el campo representa una fecha, es mejor transformarlo en características separadas de
#año, mes y día. De esta manera, la red neuronal puede capturar mejor los patrones temporales.
dataset['production_year'] = pd.to_datetime(dataset['production_date']).dt.year
dataset['production_month'] = pd.to_datetime(dataset['production_date']).dt.month
dataset['production_day'] = pd.to_datetime(dataset['production_date']).dt.day

# botar la columnar original 'production_date' 
dataset.drop(columns=['production_date'], inplace=True)

#genres: varios géneros separados por comas
#Podemos usar la codificación one-hot para convertir 
#los géneros en columnas binarias separadas para cada género.
# Convertir genres a columnas binarias utilizando one-hot encoding
dataset['genres'].replace(r'\N', '', inplace=True)
genres_list = dataset['genres'].str.get_dummies(sep=',')
dataset = pd.concat([dataset, genres_list], axis=1)

# botar la columna original 'genres'
dataset.drop(columns=['genres'], inplace=True)

#director_professions: 
#Es mejor convertir los profesiones faltantes a NaN (No es un número)
dataset['director_professions'].replace('-', np.nan, inplace=True)
#Varias profesiones separadas por coma
#De manera similar a los géneros, podemos usar la codificación one-hot para 
#convertir las profesiones de director en columnas binarias separadas para cada profesión.
professions_list = dataset['director_professions'].str.get_dummies(sep=',')
dataset = pd.concat([dataset, professions_list], axis=1)

# botar la columna original 'director_professions'
dataset.drop(columns=['director_professions'], inplace=True)

#director:birthYear: valores faltantes como '-'
#Es mejor convertir los años de nacimiento faltantes a NaN (No es un número) para que
#se manejen correctamente durante el procesamiento de datos.
dataset['director_birthYear'].replace(r'\N', '-1', inplace=True)
dataset['director_birthYear'].replace('-', -1, inplace=True)
dataset['director_birthYear'] = dataset['director_birthYear'].astype(int)

# director:deathYear: valores faltantes como '-' y 'alive' si no está muerto
#Podemos convertir los valores 'vivos' a NaN y reemplazar el '-' con NaN también.
# Convertir 'alive' a NaN 
dataset['director_deathYear'].replace('alive', -1, inplace=True)
# Convertir '-' a NaN 
dataset['director_deathYear'].replace('-', -1, inplace=True)
dataset['director_deathYear'] = dataset['director_deathYear'].astype(int)

# director_name
# Por el momento boto el nombre del director 
dataset.drop(columns=['director_name'], inplace=True)

In [97]:
dataset.columns

Index(['runtime_minutes', 'director_birthYear', 'director_deathYear',
       'movie_averageRating', 'movie_numerOfVotes', 'approval_Index',
       'Production budget $', 'Domestic gross $', 'Worldwide gross $',
       'production_year', 'production_month', 'production_day', 'Action',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Romance', 'Sci-Fi', 'Sport', 'Thriller',
       'War', 'Western', 'actor', 'actress', 'animation_department',
       'art_department', 'art_director', 'assistant_director',
       'camera_department', 'casting_department', 'casting_director',
       'cinematographer', 'composer', 'costume_designer', 'director', 'editor',
       'editorial_department', 'executive', 'location_management',
       'make_up_department', 'miscellaneous', 'music_artist',
       'music_department', 'producer', 'production_designer',
   

In [98]:
dataset.dtypes.values

array([dtype('int32'), dtype('int32'), dtype('int32'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),


In [99]:
dataset.iloc[0,:]

runtime_minutes                 192.0
director_birthYear             1954.0
director_deathYear               -1.0
movie_averageRating               7.8
movie_numerOfVotes           277543.0
                               ...   
special_effects                   0.0
stunts                            0.0
transportation_department         0.0
visual_effects                    0.0
writer                            1.0
Name: Avatar: The Way of Water, Length: 67, dtype: float64

In [100]:
# Extraer características y meta 
y = dataset['Worldwide gross $']
X = dataset.drop(columns=['Worldwide gross $'])

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identificar columnas numericas para estandarizar (excluyendo no-numericas)
numeric_columns = X_train.select_dtypes(include=['float64', 'int64','int32']).columns

# Standardize the numeric input features
scaler = StandardScaler()
X_train[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])
X_test[numeric_columns] = scaler.transform(X_test[numeric_columns])

In [101]:
# 1. Red Neuronal con Activación Sigmoidal y Regularización L1:
model_1 = tf.keras.Sequential([
    layers.Dense(64, activation='sigmoid', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='sigmoid'),
    layers.Dense(16, activation='sigmoid'),
    layers.Dense(1)
])

# Agregar regularización L1 a todas las capas ocultas
model_1.add(layers.Dense(16, activation='sigmoid', kernel_regularizer=regularizers.l1(0.01)))

# Compilar el modelo
model_1.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Entrenar el modelo
model_1.fit(X_train, y_train, epochs=20, batch_size=30, validation_data=(X_test, y_test))

Epoch 1/20
117/117 [==============================] - 1s 4ms/step - loss: 51541780805451776.0000 - mae: 111766544.0000 - val_loss: 61200135756972032.0000 - val_mae: 127529576.0000
Epoch 2/20
117/117 [==============================] - 0s 2ms/step - loss: 51541776510484480.0000 - mae: 111766552.0000 - val_loss: 61200135756972032.0000 - val_mae: 127529576.0000
Epoch 3/20
117/117 [==============================] - 0s 2ms/step - loss: 51541776510484480.0000 - mae: 111766552.0000 - val_loss: 61200135756972032.0000 - val_mae: 127529576.0000
Epoch 4/20
117/117 [==============================] - 0s 2ms/step - loss: 51541797985320960.0000 - mae: 111766544.0000 - val_loss: 61200135756972032.0000 - val_mae: 127529576.0000
Epoch 5/20
117/117 [==============================] - 0s 2ms/step - loss: 51541785100419072.0000 - mae: 111766504.0000 - val_loss: 61200135756972032.0000 - val_mae: 127529576.0000
Epoch 6/20
117/117 [==============================] - 0s 2ms/step - loss: 51541802280288256.0000 - m

In [102]:
# 2. Red Neuronal con Activación ReLU y Regularización Dropout:
model_2 = tf.keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1)
])

# Compilar el modelo
model_2.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Entrenar el modelo
model_2.fit(X_train, y_train, epochs=20, batch_size=30, validation_data=(X_test, y_test))

Epoch 1/20
117/117 [==============================] - 1s 3ms/step - loss: 51541729265844224.0000 - mae: 111766440.0000 - val_loss: 61199826519326720.0000 - val_mae: 127529016.0000
Epoch 2/20
117/117 [==============================] - 0s 2ms/step - loss: 51539311199256576.0000 - mae: 111762128.0000 - val_loss: 61191558707281920.0000 - val_mae: 127515824.0000
Epoch 3/20
117/117 [==============================] - 0s 2ms/step - loss: 51520443407925248.0000 - mae: 111731072.0000 - val_loss: 61147513817661440.0000 - val_mae: 127449240.0000
Epoch 4/20
117/117 [==============================] - 0s 2ms/step - loss: 51449108296105984.0000 - mae: 111622600.0000 - val_loss: 61011341879541760.0000 - val_mae: 127247424.0000
Epoch 5/20
117/117 [==============================] - 0s 2ms/step - loss: 51273353939386368.0000 - mae: 111358584.0000 - val_loss: 60737082447888384.0000 - val_mae: 126841776.0000
Epoch 6/20
117/117 [==============================] - 0s 2ms/step - loss: 50959091182338048.0000 - m

In [103]:
# 3. Red Neuronal con Activación Tangente Hiperbólica (Tanh) y Regularización L2:
model_3 = tf.keras.Sequential([
    layers.Dense(256, activation='tanh', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
    layers.Dense(128, activation='tanh', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(64, activation='tanh', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(1)
])

# Compilar el modelo
model_3.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Entrenar el modelo
model_3.fit(X_train, y_train, epochs=20, batch_size=30, validation_data=(X_test, y_test))

Epoch 1/20
117/117 [==============================] - 1s 4ms/step - loss: 51541785100419072.0000 - mae: 111766560.0000 - val_loss: 61200131462004736.0000 - val_mae: 127529576.0000
Epoch 2/20
117/117 [==============================] - 0s 2ms/step - loss: 51541776510484480.0000 - mae: 111766552.0000 - val_loss: 61200131462004736.0000 - val_mae: 127529576.0000
Epoch 3/20
117/117 [==============================] - 0s 2ms/step - loss: 51541772215517184.0000 - mae: 111766536.0000 - val_loss: 61200131462004736.0000 - val_mae: 127529576.0000
Epoch 4/20
117/117 [==============================] - 0s 2ms/step - loss: 51541772215517184.0000 - mae: 111766520.0000 - val_loss: 61200131462004736.0000 - val_mae: 127529568.0000
Epoch 5/20
117/117 [==============================] - 0s 2ms/step - loss: 51541785100419072.0000 - mae: 111766512.0000 - val_loss: 61200131462004736.0000 - val_mae: 127529560.0000
Epoch 6/20
117/117 [==============================] - 0s 2ms/step - loss: 51541776510484480.0000 - m

### Resultados Obtenidos

### Diferencia de rendimiento

### Red Neuronal Optima